In [4]:
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.callbacks import StreamingStdOutCallbackHandler

In [ ]:
llm = ChatOpenAI(
    temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()]
)
# Conversation memory
memory = ConversationBufferMemory(return_messages=True, memory_key="history")

# Caching for embeddings
cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

# Loading and splitting the document
loader = UnstructuredFileLoader("./files/document.txt")
docs = loader.load_and_split(text_splitter=splitter)

# Setting up embeddings with caching
embeddings = OpenAIEmbeddings()
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

# Creating the vector store and retriever
vector_store = FAISS.from_documents(docs, cached_embeddings)
retriver = vector_store.as_retriever()


### STUFF ###
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]


chain = (
    {
        "context": retriver,
        "question": RunnablePassthrough(),
        "history": RunnablePassthrough.assign(history=load_memory)["history"],
    }
    | prompt
    | llm
)

def invoke_chain(question):
    result = chain.invoke(question)
    memory.save_context({"input": question}, {"output": result.content})
    return result.content

In [ ]:
chain.invoke_chain("Is Aaronson guilty?")

In [ ]:
chain.invoke_chain("What message did he write in the table?")

In [ ]:
chain.invoke_chain("Who is Julia?")